중복되는 행과 필요없는 열을 삭제하는 일괄처리 함수 만들기

In [2]:
import pandas as pd

In [7]:
def data_cleaning(filename):
    #파일을 데이터 프레임으로 읽어옴
    ns_df = pd.read_csv(filename, encoding='EUC-KR', low_memory=False)

    #NaN인 열 삭제
    ns_book = ns_df.dropna(axis=1, how='all')

    #대출건수를 합치기 위해 필요한 행만 추출하여 count_df 데이터프레임 만들기
    count_df = ns_book[['도서명','저자','ISBN','권','대출건수']]

    #대출건수를 groupby하기
    loan_count = count_df.groupby(by = ['도서명','저자','ISBN','권'], dropna=False ).sum()

    #원본 데이터프레임에서 중복된 행을 제외하고 고유한 행만 추출하여 복사
    ns_book3 = ns_book.drop_duplicates(subset=['도서명','저자','ISBN','권']).copy()

    #도서명, 저자, ISBN, 권을 인덱스로 설정
    ns_book3.set_index(['도서명','저자','ISBN','권'], inplace=True)

    #loan_count에 저장된 누적 대출건수를 업데이트
    ns_book3.update(loan_count)

    #인덱스 재설정
    ns_book4 = ns_book3.reset_index()

    #원본 데이터프레임의 열 순서로 변경
    ns_book4 = ns_book4[ns_book.columns]
    return ns_book4

ns_book4 = data_cleaning('도서관 자료.csv')
ns_book4.info()


/var/folders/hl/v8wjcd3d7sb1xbjy562xy3400000gn/T/ipykernel_26520/1220105315.py:21: PerformanceWarning: indexing past lexsort depth may impact performance.
  ns_book3.update(loan_count)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 384591 entries, 0 to 384590
Data columns (total 13 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   번호       384591 non-null  int64 
 1   도서명      384188 non-null  object
 2   저자       384393 non-null  object
 3   출판사      379950 non-null  object
 4   발행년도     384577 non-null  object
 5   ISBN     384591 non-null  object
 6   세트 ISBN  56576 non-null   object
 7   부가기호     310386 non-null  object
 8   권        63378 non-null   object
 9   주제분류번호   364727 non-null  object
 10  도서권수     384591 non-null  int64 
 11  대출건수     384591 non-null  int64 
 12  등록일자     384591 non-null  object
dtypes: int64(3), object(10)
memory usage: 38.1+ MB


In [8]:
ns_book4.to_csv('ns_book4.csv')